In [1]:
import tensorflow as tf

C:\Users\chait\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
#from create_sentiment_features import create_feature_sets_and_labels

In [21]:
import pickle
import numpy as np

In [18]:
pickle_in = open("sentiment_set.pickle","rb")
train_x,train_y,test_x,test_y = pickle.load(pickle_in)

In [20]:
len(train_x)

9596

In [23]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 2
batch_size = 64

X = tf.placeholder('float32',[None, len(train_x[0])])
Y = tf.placeholder('float32')

# Below function is a computational graph for the neural network

def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([len(train_x[0]),n_nodes_hl1])),
                     'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1,n_nodes_hl2])),
                     'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}
    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_hl3])),
                     'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))}
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3,n_classes])),
                     'biases': tf.Variable(tf.random_normal([n_classes]))}
    # layer1
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']),hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    #Layer2
    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']),hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    #Layer3
    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']),hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    # Output Layer
    Output = tf.add(tf.matmul(l3,output_layer['weights']),output_layer['biases'])
    
    return Output

# Below function train the model
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits =prediction,labels =Y))
    
    optimizer = tf.train.AdamOptimizer().minimize(cost) # We can define learning rate also here, 
    #in this example we are taking learning rate as default 0.01
    
    hm_epochs = 10 # Giving number of epochs
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
    
        for epoch in range(hm_epochs):
            epoch_loss = 0
            
            i = 0
            while i < len(train_x):
                start = i
                end = i + batch_size
                
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                
            #for _ in range(int(mnist.train.num_examples/batch_size)):
             #   epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict = {X: batch_x,Y: batch_y})
                epoch_loss += c
                
                i+= batch_size
                
            print('Epoch', epoch, 'completed out of', hm_epochs, "loss:" ,epoch_loss)

####################################################
# Till here we trained the network
    
# Now we are testing this on test data
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(Y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy:',accuracy.eval({X:test_x, Y:test_y}))
    
train_neural_network(X)

Epoch 0 completed out of 10 loss: 319774.4569091797
Epoch 1 completed out of 10 loss: 155881.10580444336
Epoch 2 completed out of 10 loss: 90599.78912353516
Epoch 3 completed out of 10 loss: 55965.53015136719
Epoch 4 completed out of 10 loss: 42632.22664642334
Epoch 5 completed out of 10 loss: 41221.02437591553
Epoch 6 completed out of 10 loss: 47314.46713256836
Epoch 7 completed out of 10 loss: 31216.31622505188
Epoch 8 completed out of 10 loss: 24387.407606124878
Epoch 9 completed out of 10 loss: 10819.474604189396
Accuracy: 0.5694184
